In [54]:
import numpy as np
import numpy.typing as npt
import cProfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


# create a seed
seed = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.int8)

# create a biome
biome = np.zeros((100, 100), dtype=np.int8)
biome[0:10, 0:10] = seed




In [58]:
# get indices of all non-zero elements
indices = np.argwhere(biome == 1)

# get the surrounding indices
surrounding_indices = np.array([indices + np.array([i, j]) for i in range(-1, 2) for j in range(-1, 2)])

# colapse the array
surrounding_indices = surrounding_indices.reshape(-1, 2)

# remove the indices that are out of bounds
surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                          & (type(biome) == npt.NDArray[np.int8])
                                          & (surrounding_indices[:, 0] < biome.shape[0]) 
                                          & (surrounding_indices[:, 1] >= 0) 
                                          & (surrounding_indices[:, 1] < biome.shape[1])]

# remove the indices that are the same as the original indices
surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] != indices[:, 0]) | (surrounding_indices[:, 1] != indices[:, 1])]

# for each surrounding index that occures three times, set the biome to 1 
for i in np.unique(surrounding_indices, axis=0):
    if np.sum(np.all(surrounding_indices == i, axis=1)) == 3:
        biome[i[0], i[1]] = 1

# set indices that are not surrounded by 2 or 3 cells to 0
for i in np.unique(indices, axis=0):
    if np.sum(np.all(surrounding_indices == i, axis=1)) not in [2, 3]:
        biome[i[0], i[1]] = 0



SyntaxError: expected 'else' after 'if' expression (3531150994.py, line 11)

In [35]:
indices

array([[3, 3],
       [3, 4],
       [3, 5],
       [4, 5],
       [5, 4]], dtype=int64)